In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install face-recognition
clear_output()

In [ ]:
import cv2
import numpy as np
import face_recognition
from sklearn.cluster import DBSCAN
from google.colab.patches import cv2_imshow

In [ ]:
!pip install mediapipe
clear_output()

In [ ]:
import mediapipe as mp
import time
import cv2

In [ ]:
filename = 'video30.avi' 
cap = cv2.VideoCapture("./"+filename)
embeddings = []

In [ ]:
cap = cv2.VideoCapture('./'+filename)
pTime = 0
counter = 0

mpFaceDetection = mp.solutions.face_detection
mpDraw = mp.solutions.drawing_utils
faceDetection = mpFaceDetection.FaceDetection()
encodings = []
data = dict()
while True:
  success, img = cap.read()


  if success == False:
        print("Not found")
        break

  imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

 
  results = faceDetection.process(imgRGB)
  boxes = []
  ens = []
  if results.detections:
    for id, detection in enumerate(results.detections):
      bboxC = detection.location_data.relative_bounding_box
      ih, iw, ic = img.shape
      bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
             int(bboxC.width * iw), int(bboxC.height * ih)
      # print(bbox)
      encoding = face_recognition.face_encodings(img, [(bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3], bbox[0])])
      encodings += encoding
      boxes.append(tuple(bbox))

  data[counter] = {'frame': img, 'boxes': boxes, 'labels': []}
  counter += 1
  cTime = time.time()
  fps = 1 / (cTime - pTime)
  pTime = cTime
  cv2.waitKey(1)


In [ ]:
# cluster the embeddings
print("[INFO] clustering...")
clt = DBSCAN(eps=0.45, min_samples=1, metric="euclidean", n_jobs=-1)
clt.fit(encodings)
# determine the total number of unique faces found in the dataset
labelIDs = np.unique(clt.labels_)
numUniqueFaces = len(np.where(labelIDs > -1)[0])
print("[INFO] # unique faces: {}".format(numUniqueFaces))

In [ ]:
arr = list(clt.labels_)
for i in data:
  if data[i]['boxes'] != []:
    for j in data[i]['boxes']:
      data[i]['labels'].append(arr.pop(0))

In [ ]:
size = (1920, 1080)
out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)

for i in data:
  img = data[i]['frame']
  boxes = data[i]['boxes']
  labels = data[i]['labels']
  for j in range(len(boxes)):
    bbox = boxes[j]
    label = labels[j]
    cv2.rectangle(img, bbox, (255, 0, 255), 2)
    cv2.putText(img, f'{label}', (bbox[0], bbox[1]-50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)
  vidout=cv2.resize(img,size)
  out.write(vidout)

out.release()

References: https://www.youtube.com/watch?v=jn1HSXVmIrA&t=61s
https://www.pyimagesearch.com/2018/07/09/face-clustering-with-python/